In [1]:
import numpy as np
from scipy import signal
import cv2
import random as rand

In [2]:
def hadamard(n):
    if n == 1:
        return np.ones((1,1), dtype=np.int32)
    else:
        a = hadamard(n//2)
        b = np.ones((n,n), dtype=np.int32)
        sign = 1
        for i in np.arange(0,n):
            if i % 2:
                sign *= -1
            b[i,:n//2] = a[i % n//2]
            b[i,n//2:] = sign*a[i % n//2]
        return b
    
def hadamard_2d(n):
    a = hadamard(n)
    return np.array([np.outer(x,y) for x in a for y in a])

In [3]:
kernels_used_small = [0,1,9,8] #As used in the article
kernels_used_large = [0,1,2,3,4,8,9,10,11,16,17,18,24,25,32] #As used in the article: TODO find a smarter way of doing this
#Results in a lot of copy paste
def apply_wh_kernels(img, size=8):
    kernels = hadamard_2d(size) 
    #kernels used is only specified for size 8 for now.
    (x,y) = (img.shape[0] + 1 - size, img.shape[1] + 1 - size)
    y_kerns = np.ones((x,y,len(kernels_used_large)),dtype=np.int32)
    cb_kerns = np.ones((x,y,len(kernels_used_small)),dtype=np.int32)
    cr_kerns = np.ones((x,y,len(kernels_used_small)),dtype=np.int32)
    for i in np.arange(0, len(kernels_used_large)):
        y_kerns[:,:,i] = signal.convolve2d(img[:,:,0], kernels[kernels_used_large[i]], mode='valid')
    for i in np.arange(0, len(kernels_used_small)):
        cb_kerns[:,:,i] = signal.convolve2d(img[:,:,1], kernels[kernels_used_small[i]], mode='valid')
    for i in np.arange(0, len(kernels_used_small)): #might've swapped cb and cr namewise, but their use is identical/symmetrical later on, so it doesn't matter
        cr_kerns[:,:,i] = signal.convolve2d(img[:,:,2], kernels[kernels_used_small[i]], mode='valid')
    return (y_kerns, cb_kerns, cr_kerns)

In [4]:
def bin_values(img_a, img_b, bin_count, offset):
    #How this is handled is defined vaguely in the article
    rnd_nr = rand.random()
    quant_offset = rnd_nr/bin_count
    quants = [x/bin_count + quant_offset for x in np.arange(1,bin_count)]
    quants[bin_count-2] = 1
    borders = np.quantile(img_a, quants,interpolation='higher')
    return ((rnd_nr*bin_count+np.digitize(img_a, borders)) % bin_count).astype(int)<<offset, ((rnd_nr*bin_count+np.digitize(img_b, borders)) % bin_count).astype(int)<<offset
    #return borders


def create_hash_tables(img_a, img_b, L = 4, k = 2):
    (x1,y1) = (img_a[0].shape[0],img_a[0].shape[1])
    (x2,y2) = (img_b[0].shape[0],img_b[0].shape[1])
    hash_a = np.zeros((L,x1,y1),dtype = np.int64)
    hash_b = np.zeros((L,x2,y2),dtype = np.int64)
    hash_table_a = np.zeros((L,2**18,k,2),dtype = np.int64) - 1 #18 bits
    hash_table_b = np.zeros((L,2**18,k,2),dtype = np.int64) - 1 #Should've just flattened everything TODO
    #rand.random()
    magic_kernel = [0, 0, 0, 1, 5, 6, 2, 9] # kernel used
    magic_data = [0, 1, 2, 0, 0, 0, 0, 0] # colour/intensity used
    magic_bits = [5, 2, 2, 3, 3, 1, 1, 1] # number of bits for the current kernel
    #same numbers as used in article
    for i in np.arange(L):
        offset = 0
        for j in np.arange(0, 8):
            bins = 2**magic_bits[j]
            (cur_bins_a, cur_bins_b) = bin_values(img_a[magic_data[j]][:,:,magic_kernel[j]], img_b[magic_data[j]][:,:,magic_kernel[j]], bins, offset)
            hash_a[i,:,:] += cur_bins_a
            hash_b[i,:,:] += cur_bins_b
            offset+=magic_bits[j]
    print("Computed hashes")    
    #hash_tables
    for i in np.arange(L):
        for j in np.arange(x1):
            for m in np.arange(y1):
                cur_hash = hash_a[i,j,m]
                for n in np.arange(k):
                    if hash_table_a[i,cur_hash, n, 0] < 0:
                        hash_table_a[i,cur_hash, n, 0] = j
                        hash_table_a[i,cur_hash, n, 1] = m
                        break
    print("Computed hash tables for first image")
    for i in np.arange(L):
        for j in np.arange(x2):
            for m in np.arange(y2):
                cur_hash = hash_b[i,j,m]
                for n in np.arange(k):
                    if hash_table_b[i,cur_hash, n, 0] < 0:
                        hash_table_b[i,cur_hash, n, 0] = j
                        hash_table_b[i,cur_hash, n, 1] = m
                        break
    print("Computed hash tables for second image")
    return hash_a, hash_b, hash_table_a, hash_table_b


In [5]:
def init_matches(x1, y1,x2, y2):
    xs = (np.random.rand(x1,y1)*x2).astype(int)
    ys = (np.random.rand(x1,y1)*y2).astype(int)
    return xs,ys

In [6]:
def find_candidates_hash_match(hash_a, hash_table_b):
    return hash_table_b[hash_a,:,:]

def find_candidates_neighbours(hash_b, hash_table_b, cur_matches, x2, y2):
    left1,left2 = (cur_matches[0][:-1,:]+1, cur_matches[1][:-1,:])
    left1[left1==x2] = 0 # bad candidate, but current candidate is outside the patches
    left = hash_table_b[hash_b[(left1,left2)]]
    left = np.concatenate((np.zeros((1,y2,2,2),dtype=np.int64)-1,left))

    right1,right2 = (cur_matches[0][1:,:]-1, cur_matches[1][1:,:])
    right1[right1==0] = 0 # bad candidate, but current candidate is outside the patches
    right = hash_table_b[hash_b[(right1,right2)]]
    right = np.concatenate((right,np.zeros((1,y2,2,2),dtype=np.int64)-1))

    up1,up2 = (cur_matches[0][:,:-1], cur_matches[1][:,:-1]+1)
    up2[up2==y2] = 0 # bad candidate, but current candidate is outside the patches
    up = hash_table_b[hash_b[(up1,up2)]]
    up = np.concatenate((np.zeros((x2,1,2,2), dtype=np.int64)-1,up),axis=1)

    down1,down2 = (cur_matches[0][:,1:], cur_matches[1][:,1:]-1)
    down2[down2==0] = 0 # bad candidate, but current candidate is outside the patches
    down = hash_table_b[hash_b[(down1,down2)]]
    down = np.concatenate((down,np.zeros((x2,1,2,2), dtype=np.int64)-1),axis=1)

    return np.concatenate((left,right, up, down),axis=2)

def find_candidates_match_match(hash_a, hash_table_a, cur_matches):
    similar_patches = hash_table_a[hash_a,:,:]
    similar_patches[similar_patches[:,:,0] < 0] = 0
    return np.stack((cur_matches[0][similar_patches[:,:,:,0],similar_patches[:,:,:,1]],cur_matches[1][similar_patches[:,:,:,0],similar_patches[:,:,:,1]]),axis=3)
    #return similar_patches

def find_candidates(hash_a, hash_b, hash_table_a, hash_table_b, cur_matches, x2, y2):
    first_set = find_candidates_hash_match(hash_a, hash_table_b)
    second_set = find_candidates_neighbours(hash_b, hash_table_b, cur_matches, x2, y2)
    third_set = find_candidates_match_match(hash_a, hash_table_a, cur_matches)
    return np.concatenate((first_set, second_set, third_set),axis=2)


In [7]:
def find_best(candidates, kernels_a, kernels_b, x1, y1, cur_matches):
    diffs = np.zeros((x1,y1))
    diffs = np.linalg.norm(kernels_b-kernels_a, axis=2)
    cur_kernels = kernels_b[candidates[:,:,:,0], candidates[:,:,:,1]]
    cur_diffs = np.linalg.norm(cur_kernels-kernels_a[:,:,np.newaxis,:],axis=3)
    for i in np.arange(x1):
        for j in np.arange(y1):
            best = np.argmin(cur_diffs[i,j,:])
            if diffs[i,j] > cur_diffs[i,j,best]:
                cur_matches[0][i,j] = candidates[i,j,best,0]    
                cur_matches[1][i,j] = candidates[i,j,best,1]    
    return cur_matches


In [8]:
def fit(hashes, kernels_a, kernels_b):
    hash_a = hashes[0]
    kernels_a = np.concatenate((kernels_a),axis=2)
    kernels_b = np.concatenate((kernels_b),axis=2)
    hash_b = hashes[1]
    hash_table_a = hashes[2]
    hash_table_b = hashes[3]
    L,x1,y1 = hash_a.shape[0], hash_a.shape[1],hash_a.shape[2]
    x2,y2 = hash_b.shape[1],hash_b.shape[2]
    cur_matches = init_matches(x1, y1, x2, y2)
    #L = 4
    for i in np.arange(L):
        print("Starting iteration {}".format(i))
        cur_candidates = find_candidates(hash_a[i,:,:], hash_b[i,:,:], hash_table_a[i,:,:,:], hash_table_b[i,:,:,:], cur_matches, x2, y2)
        cur_matches = find_best(cur_candidates, kernels_a, kernels_b, x1, y1, cur_matches)
    return cur_matches

In [55]:
image_a = cv2.imread("data/example/1.jpg")
image_b = cv2.imread("data/example/2.jpg")

In [10]:
image_a2 = cv2.cvtColor(image_a, cv2.COLOR_BGR2YCR_CB).astype(int)
#image_b2 = cv2.cvtColor(image_b, cv2.COLOR_BGR2YCR_CB).astype(int)

In [11]:
#step indexing of CSH
image_a3 = apply_wh_kernels(image_a2)
#image_b3 = apply_wh_kernels(image_b2)

In [12]:
#step hashing of CSH
#hashes = create_hash_tables(image_a3, image_b3)
hashes = create_hash_tables(image_a3, image_a3)

Computed hashes
Computed hash tables for first image
Computed hash tables for second image


In [13]:
test = fit(hashes, image_a3, image_a3)

Starting iteration 0
Starting iteration 1
Starting iteration 2
Starting iteration 3


In [14]:
test[0].shape

(793, 1913)

In [15]:
with open('patches-self.npy', 'wb') as f:
    np.save(f, test)

In [58]:
with open('patches-self.npy', 'rb') as f:
    a= np.load(f, allow_pickle=True)

In [44]:
def mean_error(matches, orig_img, trg_img, patch_size):
    #RMS
    errs = np.zeros_like(matches[0])
    x,y,z = orig_img.shape
    xs, ys = matches
    for i in np.arange(0,patch_size):
        for j in np.arange(0,patch_size):
            errs+= np.sum((orig_img[i:x+i-patch_size+1, j:y+j-patch_size+1,:]-trg_img[xs+i,ys+j,:])**2,axis=2)
    
    return np.mean(errs/patch_size**2/3)**0.5


In [59]:
err = mean_error(a, image_a, image_a, 8)